In [7]:

# Importing libraries

import pandas as pd
import numpy as np


In [12]:
traffic = pd.read_csv("Dataset_Uber Traffic.csv")

In [13]:
print(traffic.head())
print(traffic.columns)

         DateTime  Junction  Vehicles           ID
0  1/11/2015 0:00         1        15  20151101001
1  1/11/2015 1:00         1        13  20151101011
2  1/11/2015 2:00         1        10  20151101021
3  1/11/2015 3:00         1         7  20151101031
4  1/11/2015 4:00         1         9  20151101041
Index(['DateTime', 'Junction', 'Vehicles', 'ID'], dtype='object')


In [14]:
s = traffic["DateTime"].astype(str).str.strip()

In [15]:
dt_4 = pd.to_datetime(
    s,
    format="%d/%m/%Y %H:%M",
    errors="coerce"   # rows that don't match become NaT
)

print("Parsed with 4-digit year:", dt_4.notna().sum())

Parsed with 4-digit year: 19008


In [16]:
mask = dt_4.isna()
s_bad = s[mask]

dt_2 = pd.to_datetime(
    s_bad,
    format="%d/%m/%y %H:%M",
    errors="coerce"
)

print("Parsed with 2-digit year among leftovers:", dt_2.notna().sum())

Parsed with 2-digit year among leftovers: 29112


In [18]:
dt_final = dt_4.copy()
dt_final[mask] = dt_2

print("Still missing after both passes:", dt_final.isna().sum())

Still missing after both passes: 0


In [19]:
traffic["DateTime"] = dt_final

In [20]:
traffic["DateTime"].min(), traffic["DateTime"].max()


(Timestamp('2015-11-01 00:00:00'), Timestamp('2017-06-30 23:00:00'))

In [21]:
traffic["timestamp"] = traffic["DateTime"].dt.floor("H")

C:\Users\harin\AppData\Local\Temp\ipykernel_17688\990080631.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  traffic["timestamp"] = traffic["DateTime"].dt.floor("H")


In [22]:
weather = pd.read_csv("Weather_Chennai_2015_2017_Harini.csv")
weather["timestamp"] = pd.to_datetime(weather["timestamp"])

print(traffic.shape, weather.shape)

(48120, 5) (14592, 5)


In [23]:
events = pd.read_csv("events_chennai_2015_2017_Harini.csv")

events["start"] = pd.to_datetime(events["start"])
events["end"] = pd.to_datetime(events["end"])

# Expand each event into hourly rows
rows = []
for _, r in events.iterrows():
    rng = pd.date_range(
        start=r["start"].floor("H"),
        end=r["end"].ceil("H"),
        freq="H"
    )
    for t in rng:
        rows.append({
            "timestamp": t,
            "event_name": r["event_name"],
            "event_type": r["event_type"]
        })

events_hourly = pd.DataFrame(rows).drop_duplicates()
events_hourly["event_flag"] = 1

print(events_hourly.head())


            timestamp   event_name event_type  event_flag
0 2015-11-11 00:00:00  Diwali 2015   festival           1
1 2015-11-11 01:00:00  Diwali 2015   festival           1
2 2015-11-11 02:00:00  Diwali 2015   festival           1
3 2015-11-11 03:00:00  Diwali 2015   festival           1
4 2015-11-11 04:00:00  Diwali 2015   festival           1


C:\Users\harin\AppData\Local\Temp\ipykernel_17688\3515922869.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  start=r["start"].floor("H"),
C:\Users\harin\AppData\Local\Temp\ipykernel_17688\3515922869.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end=r["end"].ceil("H"),
C:\Users\harin\AppData\Local\Temp\ipykernel_17688\3515922869.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rng = pd.date_range(


In [24]:
merged = traffic.merge(weather, on="timestamp", how="left")

print("After merging traffic+weather:", merged.shape)

After merging traffic+weather: (48120, 9)


In [25]:
merged = merged.merge(
    events_hourly[["timestamp", "event_flag", "event_type"]],
    on="timestamp",
    how="left"
)

In [26]:
merged["event_flag"] = merged["event_flag"].fillna(0).astype(int)
merged["event_type"] = merged["event_type"].fillna("none")

print("After merging events:", merged.shape)

After merging events: (48220, 11)


In [27]:
merged["date"] = merged["timestamp"].dt.date
merged["hour"] = merged["timestamp"].dt.hour
merged["dayofweek"] = merged["timestamp"].dt.dayofweek

In [28]:
merged.to_csv("Integrated_Dataset_Harini_Mukesh.csv", index=False)
print("Saved Integrated_Dataset_Harini_Mukesh.csv")

Saved Integrated_Dataset_Harini_Mukesh.csv
